In [2]:
import os

def count_images_in_folder(folder_path):
    # List of valid image extensions
    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.webp')

    # Get the list of files in the directory
    files = os.listdir(folder_path)
    
    # Count the number of images
    image_count = sum(1 for file in files if file.lower().endswith(valid_extensions))
    
    return image_count

# Path to your image directory
image_dir = './static/image'

# Get the number of images
num_images = count_images_in_folder(image_dir)

print(f"The number of images in the folder is: {num_images}")


The number of images in the folder is: 2000


In [4]:
import os
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import pandas as pd
import torch
from tqdm import tqdm

# Initialize the model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


In [5]:
device = torch.device("cpu")
model.to(device)

image_dir = './static/image'
batch_size = 8
data = []

# Function to process a batch of images
def process_batch(image_paths):
    images = [Image.open(image_path).convert("RGB") for image_path in image_paths]
    inputs = processor(images=images, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    out = model.generate(**inputs, max_new_tokens=20)
    captions = processor.batch_decode(out, skip_special_tokens=True)
    return captions

# Get list of all image paths
image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

# Process images in batches
for i in tqdm(range(0, len(image_paths), batch_size)):
    batch_paths = image_paths[i:i+batch_size]
    captions = process_batch(batch_paths)
    for img_path, caption in zip(batch_paths, captions):
        image_id = os.path.basename(img_path)
        data.append({'image_id': image_id, 'caption': caption})

df = pd.DataFrame(data)

csv_path = 'captions.csv'
df.to_csv(csv_path, index=False)

print(f"Captions saved to {csv_path}")


100%|██████████| 250/250 [1:56:18<00:00, 27.91s/it]

Captions saved to captions.csv
